In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import derivative
import math
import pandas as pd


# Рассматриваемые функции

In [ ]:
def f1(x, x2=None):
  if (x2 == None):
    return 100*(x[1] -x[0]**2)**2 + (1 - x[0])**2
  else:
    return 100*(x2 -x**2)**2 + (1 - x)**2

def f2(x, x2=None):
  if (x2 == None):
    return (x[1] - x[0]**2)**2 + (1 - x[0])**2
  else:
    return (x2 - x **2)**2 + (1 - x)**2

def f3(x, x2):
  if (x2 == None):
    return (1.5 -x[0] * (1 - x[1]))**2 + (
        2.25 -x[0] * (1 - x[1]**2))**2 + (
            2.625 - x[0] * (1 -x[1]**3))**2
  else:
    return (1.5 - x * (1 - x2))**2 +(
        2.25 -x * (1 - x2**2))**2 + (
            2.625 - x * (1 -x2**3))**2

def f4(x, x2, x3, x4):
  return (x1 +x2)**2 + 5 * (x3 - x4)**2 + (x2 - 2 * x3)**4 + 10(x1 - x4)**4

## Градиент функции в точке

In [ ]:

def partial_derivative(func, var=0, point=[]):
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = 1e-6)


In [ ]:
def grad(func, n , points):
  result = np.zeros(n)
  for i in range(n):
    result[i] = partial_derivative(func, i, points)
  return result


In [ ]:
def abs_grad(gradient):
  sum = 0
  for i in range(len(gradient)):
    sum += gradient[i]**2
  return sum**0.5

In [ ]:
''' Тестируем градиент '''
grad(f1, 2, [10,10])

array([360017.99971564, -17999.99992363])

In [ ]:



fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(1, 1, 1, projection = '3d')
x, y = np.meshgrid(np.linspace(-10, 10, 100), np.linspace(-10, 10, 100))
z = f1(x, y)

surf = ax.plot_surface(
  x, y, z, 
  rstride = 3,
  cstride = 3)

In [ ]:
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(1, 1, 1, projection = '3d')
x, y = np.meshgrid(np.linspace(-10, 10, 100), np.linspace(-10, 10, 100))
z = f2(x, y)

surf = ax.plot_surface(
  x, y, z, 
  rstride = 3,
  cstride = 3)

In [ ]:
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(1, 1, 1, projection = '3d')
x, y = np.meshgrid(np.linspace(-10, 10, 100), np.linspace(-10, 10, 100))
z = f3(x, y)

surf = ax.plot_surface(
  x, y, z, 
  rstride = 3,
  cstride = 3)

# Алгоритмы

## Метод наискорейшего спуска

In [ ]:
kz = (math.sqrt(5) + 1) / 2

def golden(func, a, b, e):
    x1 = b - (b - a) / kz
    y1 = func(x1)
    return golden_rec(func, a, b, x1, y1, e)


def golden_rec(f, left, right, x1, y1, e):
    if abs(right - left) <= e:
        return (right + left) / 2
    if x1 < (left + right) / 2:
        x2 = left + (right - left) / kz
    else:
        x2 = x1
        x1 = right - (right - left) / kz
        y1 = f(x1)
    y2 = f(x2)
    if y1 < y2:
        min_x = golden_rec(f, left, x2, x1, y1, e)
    else:
        min_x = golden_rec(f, x1, right, x2, y2, e)    
    return min_x

In [ ]:

def fast_grad(f, n, x0, e, max_iter, table):
  cur_grad = grad(f, n, x0)
  grad_value = abs_grad(cur_grad)
  n_iter = 0
  current_points = x0
  while grad_value > e and n_iter < max_iter:
    table.loc[n_iter, 'current point1'] = current_points[0]
    table.loc[n_iter, 'current point2'] = current_points[1]
    table.loc[n_iter, 'grad1'] = cur_grad[0]
    table.loc[n_iter, 'grad2'] = cur_grad[1]
    table.loc[n_iter, '|gradient|'] = grad_value

    def fi(l):
      return f(current_points - cur_grad * l)

    lm = golden(fi, -10000, 10000, 0.0000005)
    current_points -= lm * cur_grad
    table.loc[n_iter, 'lm'] =lm
    cur_grad = grad(f, n, current_points)
    grad_value = abs_grad(cur_grad)
    n_iter += 1
  return current_points





In [ ]:
def testF(x, x2 = None):
  if (x2 == None):
    return 3*x[0]**2 + x[1]**2 - x[0]*x[1] - 4 * x[0]
  return 3*x**2 + x2**2 - x*x2 -4 * x

In [ ]:
grad(testF, 2, [-2,3])

array([-19.,   8.])

In [ ]:
table = pd.DataFrame(columns=['current point1','current point2','grad1','grad2', '|gradient|', 'lm'])
fast_grad(testF, 2, [-2,3], 0.001, 20, table)


array([0.7273122, 0.3637715])

In [ ]:
table

,current point1,current point2,grad1,grad2,|gradient|,lm
0,-2,3,-19,8,20.6155,0.163587
1,1.10816,1.6913,0.957646,2.27444,2.46783,0.529928
2,0.600676,0.486012,-0.881959,0.371345,0.956948,0.163586
3,0.744954,0.425266,0.0444519,0.105575,0.114552,0.529946
4,0.721398,0.369317,-0.0409364,0.017235,0.0444166,0.163572
5,0.728095,0.366499,0.00206407,0.00490163,0.00531849,0.530287
6,0.727001,0.363901,-0.00189896,0.000798635,0.00206006,0.163269


## Метод покоординатного спуска

In [ ]:
def coordinate_descent(f, x_0, e, max_iter, table):
  diff = 1
  old_points = x_0.copy()
  current_points = x_0.copy()
  iter = 0
  while diff > e and iter < max_iter:
    for i in range(len(x_0)):
      temp_points = current_points.copy() 
      
      def fi(x):
        temp_points[i] = x
        return f(temp_points)

      new_x = golden(fi, -1000, 1000, e/4)
      current_points[i] = new_x
    
    diff = abs(f(current_points) - f(old_points))
    table.loc[iter, 'points'] = current_points
    table.loc[iter, 'delta'] = diff
    old_points = current_points.copy()
    iter += 1
  return current_points




In [ ]:
t = pd.DataFrame(columns=['points', 'delta'])
coordinate_descent(testF, [-2,3], 0.0000001, 100, t)
t

,points,delta
0,"[0.7272745019631588, 0.36363726295161103]",35.9236
1,"[0.7272745019631588, 0.36363726295161103]",0.527247
2,"[0.7272745019631588, 0.36363726295161103]",0.00366144
3,"[0.7272745019631588, 0.36363726295161103]",2.54268e-05
4,"[0.7272745019631588, 0.36363726295161103]",1.76582e-07
5,"[0.7272745019631588, 0.36363726295161103]",1.2265e-09


## Метод Брендта

In [ ]:
def brent(f, x_0, table, e=0.0001 , max_iters=10000):
  n = len(x_0)
  points = x_0.copy()
  cur_iter = 0
  delta = 0.0000000000001
  diff = e * 2
  old_value = f(points) 
  while cur_iter <= max_iters and diff > e:
    value = f(points)
    for cur_coordinate in range(n):
      new_points = points
      new_points[cur_coordinate] += delta
      new_value = f(new_points)
      d = abs((new_value - value) / delta)
      m = np.zeros(n)
      m[cur_coordinate] = 1
      f2 = lambda arg: f(points - arg * m * d)
      step = golden(f2, -100, 100, e )
      points -= step * d * m
      value = f(points)
    diff = abs(old_value - value)
    table.loc[cur_iter, 'points'] = points
    table.loc[cur_iter, 'delta'] = diff
    cur_iter += 1
    old_value = value
  return points

In [ ]:
t = pd.DataFrame(columns=['points', 'delta'])
brent(testF, [-2,3], t)
t

,points,delta
0,"[0.7275269669038622, 0.36376349946942127]",35.9237
1,"[0.7275269669038622, 0.36376349946942127]",0.527156
2,"[0.7275269669038622, 0.36376349946942127]",0.00365267
3,"[0.7275269669038622, 0.36376349946942127]",2.5357e-05


## Метод проекции градиента

In [ ]:
def projection(x):
  norm = np.linalg.norm(x)
  if norm <= 1:
        return(x)
  else:
        return(x / norm)

In [ ]:
def projection_grad(f, x0, table, e=0.0000001, max_iters=100):
  points = x0.copy()
  diff = e * 2
  cur_iter = 0
  while cur_iter < max_iters and diff > e:
    cur_iter += 1
    f2 = lambda alpha: f(points - alpha * grad(f, len(points), points))
    alpha = golden(f2, 0, 1, e)
    new_points = projection(points - alpha * grad(f, len(points), points))
    diff = abs(f(points) - f(new_points))
    points = new_points
    table.loc[cur_iter, 'points'] = points
    table.loc[cur_iter, 'delta'] = diff
  return points



In [ ]:
t = pd.DataFrame(columns=['points', 'delta'])
projection_grad(testF, [-2,3], t, 0.00001)
t

,points,delta
1,"[0.5480782472091977, 0.8364965843886938]",36.0499
2,"[0.7928439056997603, 0.609523700204398]",0.347451
3,"[0.7115842786528471, 0.3746979320728908]",0.056199
4,"[0.7285627076850908, 0.36865542620461916]",0.00101107
5,"[0.7270942199669342, 0.36382632463832515]",2.35054e-05
6,"[0.7272892270942775, 0.36374061589243933]",1.55624e-07


## Метод овражного градиента

[10, 10, 10, 10]

In [ ]:
def ravine_grad(f, x0, table, e=0.001, max_iters=1000):
  n_iter = 0
  max_iter = 200
  h = 0.5
  c = 1.5
  def fi(l):
      return y(current_points - grad * l)
  grad = nd.Gradient(y)(v0)
  v1 = []
  v1.append(v0[0] + 0.1)
  v1.append(v0[1] + 0.1)
  grad2 = nd.Gradient(y)(v1)
  current_points = v0 
  lm = golden2(fi, -10000, 10000, 0.0000005)
  x0 = v0

  x0 -= lm * grad
  lm2 = golden2(fi, -10000, 10000, 0.0000005)
  x1 = v1
  x1 -= lm2 * grad2
  current_points = v1
  v2 = x1 - (x1 - x0) * h
  grad3 = nd.Gradient(y)(v2)
  lm3 = golden2(fi, -10000, 10000, 0.0000005)
  x_2 = v2
  x_2 -= lm3 * grad3
  current_points = v2
  x_cur = x_2
  x_prev = x1
  x_prev2 = x0
  v_cur = v2
  v_prev = v1
  v_prev2 = v0
  f_x1 = y(x1)
  f_v1 = y(v1)
  def cosAlfa(u, v):
    return np.dot(u,v)/np.linalg.norm(u)/np.linalg.norm(v)

  res_grad_length = np.linalg.norm(nd.Gradient(y)(x_2))
  while res_grad_length > e and n_iter < max_iter:
    a1 = v_cur - x_prev
    a2 = v_prev - x_prev2
    b1 = x_cur - x_prev
    b2 = x_prev - x_prev2
    h = h * c ** (cosAlfa(a1, b1) - cosAlfa(a2, b2))
    v_k = x_cur - (x_cur - x_prev) * h
    grad_k = nd.Gradient(y)(v_k)
    lm_k = golden2(fi, -10000, 10000, 0.0000005)
    current_points = v_k

    x_k = v_k
    x_k -= lm_k * grad_k
    x_prev2 = x_prev
    x_prev = x_cur
    x_cur = x_k
    
    v_prev2 = v_prev
    v_prev = v_cur
    v_cur = v_k
    
    current_points = v_k
    n_iter +=1
    grad_x = nd.Gradient(y)(x_cur)
    res_grad_length = np.linalg.norm(grad_x)



In [ ]:
def fall_grad(f, x0, table, e=0.1, N=5):
  points = x0.copy()
  old_points = x0.copy()
  k = 0
  p = -grad(f, len(points), points)
  a_grad = e * 2
  cur_iter = 0
  while a_grad > e:
    k += 1
    f2 = lambda alpha: f(points - alpha * p)
    alpha = golden(f2, 0, 100, e)
    old_points = points.copy()
    points += alpha * p
    points = np.around(points, 8)
    a_grad = abs_grad(grad(f, len(x0), points))

    if k == N:
      p = -grad(f, len(x0), points)
      k = 0

    if k != 0:
      beta = np.linalg.norm(
          grad(f, len(x0), points))**2 / np.linalg.norm(
              grad(f, len(x0), old_points))**2
      p = -grad(f, len(x0), points) + beta * p
    table.loc[cur_iter, 'points'] = points
    table.loc[cur_iter, 'delta'] = a_grad
    cur_iter += 1
  return points
    

In [ ]:
t = pd.DataFrame(columns=['points', 'delta'])
fall_grad(testF, [-2,3], t)
t

,points,delta
0,"[-0.3511539328137556, 2.283758421980757]",16.0055
1,"[0.3080980350598562, 1.9472634700305491]",9.72609
2,"[0.6650583402433552, 1.7102874082853003]",5.44625
3,"[0.8550835288440148, 1.524984774856703]",3.24824
4,"[0.8695458810666772, 1.4445291149376247]",2.23005
5,"[0.8898879025640442, 1.303684916284469]",2.03226
6,"[0.9031165970083694, 1.140092459748102]",1.71785
7,"[0.9017530489592991, 0.980187242071389]",1.40497
8,"[0.8850770511786108, 0.8355971819914118]",1.14274
9,"[0.8676717968886739, 0.8067825785563117]",0.918409


In [ ]:
t.iloc[:, 0:1].values

array([[array([-0.35115393,  2.28375842])],
       [array([0.30809804, 1.94726347])],
       [array([0.66505834, 1.71028741])],
       [array([0.85508353, 1.52498477])],
       [array([0.86954588, 1.44452911])],
       [array([0.8898879 , 1.30368492])],
       [array([0.9031166 , 1.14009246])],
       [array([0.90175305, 0.98018724])],
       [array([0.88507705, 0.83559718])],
       [array([0.8676718 , 0.80678258])],
       [array([0.83826795, 0.75499048])],
       [array([0.80656841, 0.69226323])],
       [array([0.77976474, 0.62871116])],
       [array([0.76055663, 0.56999332])],
       [array([0.76080234, 0.5560866 ])],
       [array([0.76069412, 0.5312781 ])],
       [array([0.75940822, 0.50169788])],
       [array([0.75652978, 0.47269525])],
       [array([0.752248  , 0.44717922])],
       [array([0.74981914, 0.4419719 ])],
       [array([0.7456358 , 0.43256093])],
       [array([0.74097588, 0.42109967])],
       [array([0.73690244, 0.40960177])],
       [array([0.73690544, 0.409

In [ ]:
def plot_func(f, )